This notebook identifies variables to be excluded from the training data set. These variables have been excluded for reasons such as being non-relevant identifiers, components of the target variable, or data from after mortgage origination providing data leakage.

Variables will be stored in .yaml file and utilized by the train_test_split.py script

In [67]:
import re
import yaml
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [68]:
df = pd.read_csv('bin/cleaned_and_joined.csv')

In [69]:
# List variables to be excluded

exclude_variables = ['nsmoid',          # NSMO Identification Number
                     'survey_wave',     # NSMO Survey Wave (Quarterly)
                     'analysis_weight', # NSMO Analysis Weight (Sampling Weight x Non-response Adjustment)
                     'rate_spread',     # Mortgage Interest Rate Spread at Origination (Percent)
                     'pmms',            # Freddie Mac's Primary Mortgage Market Survey (PMMS) Rate at Origination (Percent)
                     'DGS30',           # Market Yield on U.S. Treasury Securities at 30-Year Constant Maturity, Quoted on an Investment Basis
                     'Beta'             # Original Beta before it was winsorized
                    ]

In [70]:
# Identify date columns. These contain future information that can introduce data leakage to a model

# Define regex patterns
pattern_4digits = re.compile(r'\d{4}$')               # ends in 4 digits
pattern_4digits_letter = re.compile(r'\d{4}_[A-Za-z0-9]$')  # ends in '####_[alphanumeric character]'

# Filter matching columns
date_columns = [
    col for col in df.columns 
    if pattern_4digits.search(col) or pattern_4digits_letter.search(col)
]

print(date_columns)

['score_0313_r', 'score_0613_r', 'score_0913_r', 'score_1213_r', 'score_0314_r', 'score_0614_r', 'score_0914_r', 'score_1214_r', 'score_0315_r', 'score_0615_r', 'score_0915_r', 'score_1215_r', 'score_0316_r', 'score_0616_r', 'score_0916_r', 'score_1216_r', 'score_0317_r', 'score_0617_r', 'score_0917_r', 'score_1217_r', 'score_0318_r', 'score_0618_r', 'score_0918_r', 'score_1218_r', 'score_0319_r', 'score_0619_r', 'score_0919_r', 'score_1219_r', 'score_0320_r', 'score_0620_r', 'score_0920_r', 'score_1220_r', 'score_0321_r', 'score_0621_r', 'score_0921_r', 'score_1221_r', 'score_0322_r', 'score_0622_r', 'score_0922_r', 'score_1222_r', 'score_0323_r', 'score_0623_r', 'score_0923_r', 'score_0313_s', 'score_0613_s', 'score_0913_s', 'score_1213_s', 'score_0314_s', 'score_0614_s', 'score_0914_s', 'score_1214_s', 'score_0315_s', 'score_0615_s', 'score_0915_s', 'score_1215_s', 'score_0316_s', 'score_0616_s', 'score_0916_s', 'score_1216_s', 'score_0317_s', 'score_0617_s', 'score_0917_s', 'score_

In [71]:
# Combine the two lists of variables
exclude_variables += date_columns

In [72]:
df = df.drop(exclude_variables, axis=1)

In [73]:
# Survey question columns being dropped for pure calcluation based on borrower profile
survey_columns = [col for col in df.columns if col.startswith('x')]

In [74]:
# Demographic data in x74 - 80, keeping those columns
demographic_columns = ['x' + str(i) for i in range(74,81)]

In [75]:
survey_columns = [col for col in survey_columns if not col.startswith(tuple(demographic_columns))]

In [76]:
df = df.drop(survey_columns, axis=1)
exclude_variables += survey_columns

In [77]:
# Survey indicator columns being dropped
indicator_columns = [col for col in df.columns if col.startswith('z')]

In [78]:
df = df.drop(indicator_columns, axis=1)
exclude_variables += indicator_columns

In [79]:
# Identifying low score, as banks typically use "lower middle" scoring
# Banks pull 3 bureaus for each borrower, take the middle score for each borrower
# then use the lowest borrowers middle score
score_columns = [col for col in df.columns if col.startswith('score')]

In [80]:
score_columns

['score_orig_r',
 'score_orig_s',
 'score_orig_o1',
 'score_orig_o2',
 'score_orig_o3']

In [81]:
df['low_score'] = df[score_columns].min(axis=1)

In [82]:
df['low_score'].describe()

count   50542.000
mean      739.646
std        64.210
min       422.000
25%       697.000
50%       752.000
75%       794.000
max       839.000
Name: low_score, dtype: float64

In [83]:
df = df.drop(score_columns, axis=1)
exclude_variables += score_columns

In [84]:
# dropping open/close month columns as this contains possible data leakage of interest rates
open_columns = [col for col in df.columns if col.startswith('open')]
close_columns = [col for col in df.columns if col.startswith('close')]

In [85]:
open_columns

['open_month_1',
 'open_month_2',
 'open_month_3',
 'open_month_4',
 'open_month_5',
 'open_month_6',
 'open_month_7',
 'open_month_8',
 'open_month_9',
 'open_month_10',
 'open_month_11',
 'open_month_12']

In [86]:
close_columns

['close_month_1',
 'close_month_2',
 'close_month_3',
 'close_month_4',
 'close_month_5',
 'close_month_6',
 'close_month_7',
 'close_month_8',
 'close_month_9',
 'close_month_10',
 'close_month_11',
 'close_month_12']

In [87]:
df = df.drop(open_columns + close_columns, axis=1)
exclude_variables += open_columns + close_columns

In [88]:
with open('model_excluded_variables.yaml', 'w', encoding='utf-8') as file:
    yaml.dump(exclude_variables, file)